# Optimization Tool for Shipment Quantities

### Problem: We are often asked to quote prices based on which products and how much quantities our customers want. There are many factors to consider before making the decision, but currently we do not have an easy method to solve it.

### This is a tool that helps us to find the optimal quantities of requested items for shipment. Below are what need to be considered:

- The total volume must not exceed a container's capacity (containers: 20ft, 40ft, 40hq).  
- There might be some production capacity for certain products, so the shipment quantity cannot go over that.  
- 'Demand' indicates what the customer is requesting for quote. This tool will adjust quantities from this 'Demand' while keeping the constraints and objective.
- The objective is to maximize the profit.

In [ ]:
import numpy as np
import pandas as pd
import pulp as pl

#### Below DataFrame contains the volume (CBM), unit profit, and quantity per box (in yards) for each item.

In [81]:
df_box = pd.read_excel("box_sizes.xlsx")
df_box['CBM'] = df_box['Width'] * df_box['Length'] * df_box['Height']
df_box.head()

,Item,Per Box,Width,Length,Height,Unit Profit,CBM
0,Satin Taffeta K-1479,250,1.51,0.21,0.51,0.93,0.161721
1,Taffeta K-2790,250,0.80,0.19,0.33,0.84,0.050160
2,Hi Multi Chiffon K-711,500,1.52,0.23,0.57,0.89,0.199272
3,Rendezvous Chiffon K-2420,500,1.52,0.25,0.61,0.92,0.231800
4,Knit Chiffon K-2108,500,1.52,0.28,0.70,0.60,0.297920


#### Input data: item, demand, and production capacity  
* If there is no capacity limit, put a big number (> 100,000)

In [128]:
import ipywidgets as widgets
from IPython.display import display

# Dropdown for item selection
dropdown = widgets.Dropdown(
    options=df_box['Item'].dropna().unique(),
    description='Select Item:',
    disabled=False,
)

# Input boxes for demand and capacity
demand_box = widgets.IntText(description='Demand:')
capacity_box = widgets.IntText(description='Capacity:')
save_button = widgets.Button(description='Save Data')

# List to store the selections along with demand and capacity
selections = []

# Variable to keep track of the currently selected item
current_item = None

# Function to handle changes in the dropdown
def dropdown_eventhandler(change):
    global current_item
    current_item = change.new

# Function to handle saving the data
def save_data(b):
    if current_item is not None:
        # Check if the item already exists in the selections
        existing_entry = next((entry for entry in selections if entry['item'] == current_item), None)
        
        if existing_entry:
            # Update the existing entry
            existing_entry['demand'] = demand_box.value
            existing_entry['capacity'] = capacity_box.value
        else:
            # Add a new entry
            selections.append({'item': current_item, 'demand': demand_box.value, 'capacity': capacity_box.value})
        
        print(f"Current selections: {selections}")

# Set up event handlers
dropdown.observe(dropdown_eventhandler, names='value')
save_button.on_click(save_data)

# Display the widgets
display(dropdown, demand_box, capacity_box, save_button)

Dropdown(description='Select Item:', options=('Satin Taffeta K-1479 ', 'Taffeta K-2790 ', 'Hi Multi Chiffon K-…

IntText(value=0, description='Demand:')

IntText(value=0, description='Capacity:')

Button(description='Save Data', style=ButtonStyle())

Current selections: [{'item': 'Hi Multi Chiffon K-711 ', 'demand': 50000, 'capacity': 100000}]
Current selections: [{'item': 'Hi Multi Chiffon K-711 ', 'demand': 50000, 'capacity': 100000}, {'item': 'Satin Taffeta K-1479 ', 'demand': 30000, 'capacity': 100000}]
Current selections: [{'item': 'Hi Multi Chiffon K-711 ', 'demand': 50000, 'capacity': 100000}, {'item': 'Satin Taffeta K-1479 ', 'demand': 30000, 'capacity': 100000}, {'item': 'Velvet  (320G)', 'demand': 20000, 'capacity': 100000}]
Current selections: [{'item': 'Hi Multi Chiffon K-711 ', 'demand': 50000, 'capacity': 100000}, {'item': 'Satin Taffeta K-1479 ', 'demand': 45000, 'capacity': 100000}, {'item': 'Velvet  (320G)', 'demand': 20000, 'capacity': 100000}]
Current selections: [{'item': 'Hi Multi Chiffon K-711 ', 'demand': 50000, 'capacity': 100000}, {'item': 'Satin Taffeta K-1479 ', 'demand': 45000, 'capacity': 100000}, {'item': 'Velvet  (320G)', 'demand': 20000, 'capacity': 100000}, {'item': 'Poly Tull ', 'demand': 10000, 'c

#### Select container size

In [206]:
import ipywidgets as widgets
from IPython.display import display

# Dropdown for container selection
container_dropdown = widgets.Dropdown(
    options=['20ft', '40ft', '40hq'],
    description='Container:',
    disabled=False,
)

# Variable to store the selected container type
selected_container = None

# Function to handle changes in the dropdown
def dropdown_eventhandler(change):
    global selected_container
    selected_container = change.new
    print(f"Selected container: {selected_container}")

# Observe function for the dropdown
container_dropdown.observe(dropdown_eventhandler, names='value')

# Display the dropdown
display(container_dropdown)

Dropdown(description='Container:', options=('20ft', '40ft', '40hq'), value='20ft')

Selected container: 40hq


In [207]:
df_selected = pd.DataFrame(selections)
df_selected

,item,demand,capacity
0,Hi Multi Chiffon K-711,50000,45000
1,Satin Taffeta K-1479,45000,100000
2,Velvet (320G),20000,100000
3,Poly Tull,10000,100000


In [208]:
selected_container

'40hq'

In [209]:
# Dimensions for each container size
dim_20ft = (2.33, 5.9, 2.35)
dim_40ft = (2.33, 11.99, 2.35)
dim_40hq = (2.33, 11.99, 2.68)

# Volume for each container size
cbm_20ft = dim_20ft[0] * dim_20ft[1] * dim_20ft[2]
cbm_40ft = dim_40ft[0] * dim_40ft[1] * dim_40ft[2]
cbm_40hq = dim_40hq[0] * dim_40hq[1] * dim_40hq[2]

In [210]:
df_merged = df_selected.merge(df_box, left_on='item', right_on='Item')
df_merged = df_merged.drop(['Item', 'Width', 'Length', 'Height'], axis=1)
df_merged

,item,demand,capacity,Per Box,Unit Profit,CBM
0,Hi Multi Chiffon K-711,50000,45000,500,0.89,0.199272
1,Satin Taffeta K-1479,45000,100000,250,0.93,0.161721
2,Velvet (320G),20000,100000,150,0.88,0.209250
3,Poly Tull,10000,100000,500,0.68,0.224532


In [211]:
m = pl.LpProblem('Textiles', pl.LpMaximize)

In [212]:
# Create variables
x = []
for i in df_merged.index:
    x.append(pl.LpVariable(f'q_{i}', cat='Integer') )
    
x = np.array(x)

In [213]:
# Create constraints

# Production for each item has a capacity constraint
for i in df_merged.index:
    m += (x[i] <= df_merged.capacity[i], f'Production capacity constraint {i}')

# Must produce at least as much as demand
for i in df_merged.index:
    m += (x[i] >= df_merged.demand[i] - 0.2 * df_merged.demand[i], f'Item {i} minimum constraint')
    m += (x[i] <= df_merged.demand[i] + 0.2 * df_merged.demand[i], f'Item {i} maximum constraint')

# Total volume of all boxes must be less than 0.9 times volume of container
m += (pl.lpSum(np.array(df_merged['CBM']) * x / df_merged['Per Box']) <= 0.9 * cbm_40hq)

In [214]:
m += pl.lpSum(np.array(df_merged['Unit Profit']) * x)

In [215]:
m

Textiles:
MAXIMIZE
0.89*q_0 + 0.93*q_1 + 0.88*q_2 + 0.68*q_3 + 0.0
SUBJECT TO
Production_capacity_constraint_0: q_0 <= 45000

Production_capacity_constraint_1: q_1 <= 100000

Production_capacity_constraint_2: q_2 <= 100000

Production_capacity_constraint_3: q_3 <= 100000

Item_0_minimum_constraint: q_0 >= 40000

Item_0_maximum_constraint: q_0 <= 60000

Item_1_minimum_constraint: q_1 >= 36000

Item_1_maximum_constraint: q_1 <= 54000

Item_2_minimum_constraint: q_2 >= 16000

Item_2_maximum_constraint: q_2 <= 24000

Item_3_minimum_constraint: q_3 >= 8000

Item_3_maximum_constraint: q_3 <= 12000

_C1: 0.000398544 q_0 + 0.000646884 q_1 + 0.001395 q_2 + 0.000449064 q_3
 <= 67.3833204

VARIABLES
q_0 free Integer
q_1 free Integer
q_2 free Integer
q_3 free Integer

In [216]:
# Solve the linear optimization problem
result = pl.PULP_CBC_CMD().solve(m)

if result == 1:
    print("Shipment accepted")
else:
    print("Shipment not accepted")

Shipment accepted


In [217]:
# Obtain optimal solution
optimal_soln = m.objective.value()
optimal_soln

93426.11000000002

In [221]:
df_merged['Production'] = [x[i].value() for i in range(len(x))]
df_merged

,item,demand,capacity,Per Box,Unit Profit,CBM,Production
0,Hi Multi Chiffon K-711,50000,45000,500,0.89,0.199272,45000.0
1,Satin Taffeta K-1479,45000,100000,250,0.93,0.161721,36003.0
2,Velvet (320G),20000,100000,150,0.88,0.209250,16000.0
3,Poly Tull,10000,100000,500,0.68,0.224532,8549.0


In [222]:
# Round down to thousands because orders are usually made in thousands and rounding up can cause excess capacity
df_merged['Production'] = ((df_merged['Production'] // 1000) * 1000).astype(int)

In [223]:
df_merged

,item,demand,capacity,Per Box,Unit Profit,CBM,Production
0,Hi Multi Chiffon K-711,50000,45000,500,0.89,0.199272,45000
1,Satin Taffeta K-1479,45000,100000,250,0.93,0.161721,36000
2,Velvet (320G),20000,100000,150,0.88,0.209250,16000
3,Poly Tull,10000,100000,500,0.68,0.224532,8000


### 'Production' column shows the optimal quantities we can sell for this customer.